In [2]:
import pandas as pd

In [5]:
alternatives = pd.read_csv("./data/milosz_alternatives1.csv")
criteria = pd.read_csv("./data/milosz_criteria1.csv")

In [6]:
alternatives

,country,power,safety,construction_cost
0,ITA,98,8,400
1,BEL,58,0,800
2,GER,66,5,1000
3,SWE,74,3,600
4,AUT,80,7,200
5,FRA,82,10,600


In [7]:
criteria

,criterion_name,criterion_type,indifference,preference,veto,weight
0,power,gain,0,0.000001,NaN,3
1,safety,gain,0,2.000000,NaN,2
2,construction_cost,cost,100,300.000000,NaN,5


In [15]:
from validate_input import validate_input
validate_input(alternatives, criteria)

True

In [6]:
print(criteria.iloc[0])

criterion_name       power
criterion_type        gain
indifference             0
preference        0.000001
veto                   NaN
weight                   3
Name: 0, dtype: object


In [19]:
# compute preference between alternatives a and b
def marginal_preference_index(a, b, criterion):
    p = criterion["preference"]
    q = criterion["indifference"]
    criterion_type = criterion["criterion_type"]
    criterion_name = criterion["criterion_name"]
    a_val = a[criterion_name]
    b_val = b[criterion_name]
    difference = a_val - b_val
    if criterion_type == "cost":
        difference *= -1

    if difference > p:
        return 1
    if difference <= q:
        return 0
    return (difference - q) / (p-q)

In [30]:
def comprehensive_preference_index(a, b, criteria):
    numerator_sum = sum([marginal_preference_index(a, b, criterion[1])*criterion[1]["weight"] for criterion in criteria.iterrows()])
    sum_of_weights = sum([criterion[1]["weight"] for criterion in criteria.iterrows()])
    return numerator_sum/sum_of_weights

In [34]:
# different results for FRA-AUT and AUT-FRA than in the presentation
# manual verification suggests that Milosz made an oopsie
for alternative in alternatives.iterrows():
    for b_alternative in alternatives.iterrows():
        if alternative[0] == b_alternative[0]:
            continue
        
        print(alternative[1][0], b_alternative[1][0], comprehensive_preference_index(alternative[1], b_alternative[1], criteria))

ITA BEL 1.0
ITA GER 1.0
ITA SWE 0.75
ITA AUT 0.4
ITA FRA 0.55
BEL ITA 0.0
BEL GER 0.25
BEL SWE 0.0
BEL AUT 0.0
BEL FRA 0.0
GER ITA 0.0
GER BEL 0.5
GER SWE 0.2
GER AUT 0.0
GER FRA 0.0
SWE ITA 0.0
SWE BEL 0.75
SWE GER 0.8
SWE AUT 0.0
SWE FRA 0.0
AUT ITA 0.25
AUT BEL 1.0
AUT GER 1.0
AUT SWE 1.0
AUT FRA 0.5
FRA ITA 0.2
FRA BEL 0.75
FRA GER 1.0
FRA SWE 0.5
FRA AUT 0.5
